<a href="https://colab.research.google.com/github/sp8rks/MaterialsInformatics/blob/main/worked_examples/RDKit_tutorial/RDkit_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RDKIT overview

To use this notebook make sure to download RDKIT with conda install rdkit

Google Colab link https://colab.research.google.com/drive/1hy0wXH7yFKoXjGVPWI098yxvV4AKsabL?usp=sharing

# major types of moleculer file formats
* SMILES -- plain text (string) [more description ](http://www.daylight.com/dayhtml/doc/theory/theory.smiles.html)
* sdf    -- plain text (acom coordinates)

## SMILES

Simplest way to reflect a molecule:
* atoms of chemical elements are represented by chemical symbols in capital letter, hydrogen is usually ignored. 
* bonds :
    * single bonds are not displayed 
    * for double '=' symbol
    * for triple '#' symbol
    * for quadruple '$' symbol 
* atoms that are bonded must stand nearby.
Ring structures are written by breaking each ring at an arbitrary point (although some choices will lead to a more legible SMILES than others) to make a 'straight non-ring' structure (as if it wasn't a ring) and adding numerical ring closure labels to show connectivity between non-adjacent atoms.
* aromacity is commonly illustrated by writing the constituent B, C, N, O, P and S atoms in lower-case forms b, c, n, o, p and s, respectively.
* for side chains representation of atomic gpoups branches are used.

pro :
* easy to write
* easy to store

cons:
* (usually) does not have a 3D information
* one molecule could have multiple (!) smiles

#### Video

https://www.youtube.com/watch?v=kBk8HbjWwCw&list=PLL0SWcFqypCl4lrzk1dMWwTUrzQZFt7y0&index=15 (Molecular Strings and Fingerprints RDKit tutorial)

# Importing data to RDKIT

rdkit have it's inner molecular data representation object (called mol object)

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit import DataStructs

# Data

Read the smiles data and load it into a dataframe. Index out the first SMILEs string and read it.

In [ ]:
data = pd.read_csv("full_dataset_dd.csv",index_col=0)
data.head()

In [ ]:
my_one_smile_string = data["SMILES_STD"][0]
print(my_one_smile_string)

Creates a mol object from a SMILEs string. 

In [ ]:
my_one_mol_object = Chem.MolFromSmiles(my_one_smile_string,sanitize=True)

In [ ]:
type(my_one_mol_object)

In [ ]:
my_one_mol_object

Convert a mole object to a SMILEs string. Mol objects are useful because, unlike SMILEs strings, they can be rendered as 2D and 3D objects. 

In [ ]:
Chem.MolToSmiles(my_one_mol_object)

In [ ]:
Chem.MolToInchiKey(my_one_mol_object)

In [ ]:
my_one_mol_object.GetNumAtoms()

Converting a mol object to a mol block can be useful as the block holds the atomic coordinates of the atoms in the molecule. This lets them hold 3D information about the molecule in question. 

In [ ]:
# Molecule representation
#https://en.wikipedia.org/wiki/Chemical_table_file#Extended_Connection_Table_(V3000)
print(Chem.MolToMolBlock(my_one_mol_object))

In [ ]:
from rdkit.Chem import AllChem
my_one_mol_object_3D = Chem.AddHs(my_one_mol_object) # by default rdkit's mol object does not store H's
AllChem.EmbedMolecule(my_one_mol_object_3D)

In [ ]:
my_one_mol_object_3D

In [ ]:
print(Chem.MolToMolBlock(my_one_mol_object_3D))

## Write a molecule to an sdf file

In [ ]:
w = Chem.SDWriter('./my_one_mol_object_3D.sdf')
w.write(my_one_mol_object_3D)
w.close()

Works on linux to view the sdf file

In [ ]:
%%bash
less ./my_one_mol_object_3D.sdf

# Working with pandas dataframes

You could work with help of PandasTools module (easy to use, good for "small" data).
Or you could do everything on your own, which is good when trying to deal with "big" data (more than 100K molecules).

In [ ]:
data_small = data.head(40).copy()

In [ ]:
data_small.head()

In [ ]:
from rdkit.Chem import PandasTools
PandasTools.AddMoleculeColumnToFrame(data_small, smilesCol='SMILES_STD',molCol='ROMol')


In [ ]:
data_small.head()

**Important to know** ROMol is not a column with pictures, its column with Mol Objects, represented as pictures

In [ ]:
# manual way
data_small["MolObj"] = data_small['SMILES_STD'].map(lambda x: Chem.MolFromSmiles(x))

In [ ]:
data_small.head()

Why do we need a manual way? Store Mol Objects **in memory could be hard**. so sometimes its useful to perform calculation of interest **on the fly**

In [ ]:
# example of cacluclating n_atoms without storing MolObjects
data_small["n_Atoms"] = data_small['SMILES_STD'].map(lambda x: Chem.MolFromSmiles(x).GetNumAtoms())

In [ ]:
data_small.head()

## Drawing a mol grid

In [ ]:
from rdkit.Chem import Draw
Draw.MolsToGridImage(list(data_small.ROMol), useSVG=True)

# Molecular fingerprints

Molecular fingerprints are useful when trying to conduct similarity searches and other comparison tasks. This is useful when trying to perform classification and clustering.

![img](./fp_list.png)

In [ ]:
my_this_mol_obj = data_small["ROMol"][0]
my_that_mol_obj = data_small["ROMol"][1]

## Path-Based Fingerprint

This type of fingerprint encodes information about the specific linear paths within a molecule. As the path length increases so does the chunk of atoms considered. Path Eneumeration shows all possible linear paths in the molecule up to a specific length. In the example below a maxPath of 7 means that the fingerprint will consider paths with up to 7 bonds. Each path is differentiated by a sequence of atoms and bond types along the path. 

In [ ]:
this_fp = Chem.RDKFingerprint(my_this_mol_obj,maxPath=7,fpSize=512)
that_fp = Chem.RDKFingerprint(my_that_mol_obj,maxPath=7,fpSize=512)

In [ ]:
this_fp

In [ ]:
that_fp.ToBitString()

In [ ]:
import numpy as np
np.array(that_fp)

To see how path length changes the fingerprint, see this illustration from OpenEye documentation


![pathfp](./PathEnumeration.png)

## Morgan Fingerprint

The Morgan Fingerprint is a type of molecular fingerprint used in cheminformatics to represent the structure of molecules. It's a circular fingerprint, and describes the environment around each atom within a specified radius. For example, a radius of 2 means that it will look at the atom, its neighbors, and the neighbors' neighbors. In the generated image below you can see that as the circular radius increases, so does the chunk of atoms considered. 

In [ ]:
# Morgan FP
MFP_this = AllChem.GetMorganFingerprint(my_this_mol_obj,2)
MFP_this

To see how changing the circular radius changes the fingerprint, see this illustration from OpenEye documentation

 
![pathfp](./CircularEnumeration.png)

Get the Morgan Fingerprint as a bit vector

In [ ]:
MFP_this_bits = AllChem.GetMorganFingerprintAsBitVect(my_this_mol_obj,5,nBits=512)
MFP_this_bits

In [ ]:
MFP_this_bits.ToBitString()

Calculate the dice similarities betwen the two fingerprint bit strings.

In [ ]:
DataStructs.DiceSimilarity(this_fp,this_fp)

In [ ]:
DataStructs.DiceSimilarity(this_fp,that_fp)

In [ ]:
DataStructs.DiceSimilarity(this_fp,MFP_this_bits)

## What does the bit mean?

A bit refers to a binary digit that can encode information about the presence or absence of certain substructures or features of the molecule. 

In [ ]:
from rdkit.Chem import Draw
mol = Chem.MolFromSmiles('c1ccccc1CC1CC1')
bi = {}
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, bitInfo=bi,nBits=512)

mfp2_svg = Draw.DrawMorganBit(mol, 29, bi, useSVG=True)
rdkbi = {}
rdkfp = Chem.RDKFingerprint(mol, maxPath=5, bitInfo=rdkbi,fpSize=512)
rdk_svg = Draw.DrawRDKitBit(mol, 2, rdkbi, useSVG=True)

In [ ]:
mfp2_svg

In [ ]:
rdk_svg

## Molecular similarity

Molecular similarity measures how simliar two molecules are to one another based off various criteria. These can be structural features, functional groups, chemical properties, etc. Using molecular similarity allows us to virtually screen for molecules within a certain system or sharing properties. This is helpful when trying to discover new compounds, as we can use previous clustering of compounds to find ones with simliar properties. 

## Dice similarity

Dice similarity is a specific metric used to compare the similarity between two sets. This can be extended to molecular fingerprints. Dice similarity is especially useful when trying to compare two binary vectors (such as chemical fingerprints!). Dice similarity is a subset of molecular similarity.

![dice](./dice_sim.png)

## Tanimoto similarity 

The Tanimoto similarity is also used to compare two binary vectors of molecular fingerprints. The value of the similarity falls between 0 and 1 (with 0 being no similarity and 1 being all similarity).

![tani](./jacard_sim.jpg)

In [ ]:
from rdkit import DataStructs
# Tanimoto
commonBits = this_fp&that_fp
print('this:',this_fp.GetNumOnBits(),'that:',that_fp.GetNumOnBits(),'num in common:',commonBits.GetNumOnBits())
print(commonBits.GetNumOnBits()/(this_fp.GetNumOnBits()+that_fp.GetNumOnBits()-commonBits.GetNumOnBits()))
print('Tanimoto:', DataStructs.TanimotoSimilarity(this_fp,that_fp))

In [ ]:
from sklearn.metrics import jaccard_score

In [ ]:
jaccard_score(np.array(this_fp),np.array(that_fp))

## Molecular similarity visualization 

This example generates visualizations that highlight the similarities and differences between two molecules based off their found fingerprints. This is a good way to visualize which parts contribute the most to their similarity and dissimilarity. This example is specifically using a Morgan Fingerpring and a Tanimoto Similarity to compare two molecules. 

In [ ]:
from rdkit.Chem.Draw import SimilarityMaps

## Interpretation

The two images below (with the shaded colors on them) are the two molecules we are comparing. The intensity of the color determines how much each atom contributes either negatively or positively to the similarity between the two molecules. The red atoms signify a negative contribution to the similarity (as seen on the left in the first picture) and green signifies a positive contribution to the similarity. This finger print is using a circular radius of 1 for the Morgan Fingerprint and so will only consider a single atom at a time. 

In [ ]:
fig, maxweight = SimilarityMaps.GetSimilarityMapForFingerprint(my_this_mol_obj,my_that_mol_obj, SimilarityMaps.GetMorganFingerprint)
fig, maxweight = SimilarityMaps.GetSimilarityMapForFingerprint(my_that_mol_obj,my_this_mol_obj, SimilarityMaps.GetMorganFingerprint)

In [ ]:
Draw.MolsToGridImage([my_this_mol_obj,my_that_mol_obj])

Now we are using a circular radius of 5 which allows the fingerprint to consider much larger groups of atoms. This can be important to visualize the contributions of functional groups as a whole rather than individual parts. 

In [ ]:
fig, maxweight = SimilarityMaps.GetSimilarityMapForFingerprint(my_this_mol_obj, my_that_mol_obj,
                                                               lambda m,idx: SimilarityMaps.GetMorganFingerprint(m, atomId=idx, radius=5, fpType='bv'),
                                                               metric=DataStructs.TanimotoSimilarity)


## On a bigger scale with pandas dataframe 

Now we will compare molecules within a pandas dataframe rather than one at a time. 

In [ ]:
def SMI2FD(smiles_str,mP=7,fS=1024):
    fp = Chem.RDKFingerprint(Chem.MolFromSmiles(smiles_str),maxPath=mP,fpSize=fS)
    return np.array(fp)
data_small["RDKFP"]=data_small["SMILES_STD"].apply(SMI2FD)

In [ ]:
data_small.head(1)

In [ ]:
first_fp = data_small["RDKFP"][0]

In [ ]:
data_small["Tanimoto_sim_toFirst"]=data_small["RDKFP"].apply(lambda x: jaccard_score(x,first_fp))

In [ ]:
data_small.sort_values(['Tanimoto_sim_toFirst'], inplace=False, ascending=False)

## Descriptors

![desc](./de_list.png)

# Descriptors

This code calculates molecular descriptors, generates simliarity maps, and computes the distance between molecular descriptors. 

The TPSA (Topological Polar Surface Area) descriptor sums the surface areas of polar atoms in a molecule. This can be helpful when trying to predict drug absorption and permeability. 

The MolLogP descriptor finds the LogP value (octanol-water partition coefficient) which is a measure of a molecule's hydrophobicity. It's often used to predict the solubility and permeability of drugs. 

In [ ]:
from rdkit.Chem import Descriptors
print("TPSA %s"%Descriptors.TPSA(my_this_mol_obj))
print("mollogP %s"%Descriptors.MolLogP(my_this_mol_obj))

With rdMolDescriptors these descriptors can easilly be visualized on the actual molecule. The CalcCrippenContribs shows this as an image ad the similarity map shows the contributions of each atom as a shaded region. 

In [ ]:
from rdkit.Chem import rdMolDescriptors
contribs = rdMolDescriptors._CalcCrippenContribs(my_this_mol_obj)
fig = SimilarityMaps.GetSimilarityMapFromWeights(my_this_mol_obj,[x for x,y in contribs], colorMap='jet', contourLines=10)


This code allows us to calculate all the molecular descriptors that RDKit has to offer. 

In [ ]:

from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
nms=[x[0] for x in Descriptors._descList]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(nms)
print(len(nms))

In [ ]:
my_precious_descs = [calc.CalcDescriptors(x) for x in [my_this_mol_obj,my_that_mol_obj]]
my_precious_descs

In [ ]:
np.array(my_precious_descs).shape

The Euclidian Distance between each set of descriptors quantifies how simliar or different the two molecules are based off their descriptors. This is useful as it allows us to find the similarity between two molecules from multiple descriptors. Additionally, descriptors can be applied to a wide range of properties and so allows us to compare them in more ways than just atom connectivity (for example, the similarity of two molecules based off of TPSA AND molLogP)

In [ ]:
from sklearn.metrics import euclidean_distances
euclidean_distances(np.array(my_precious_descs))

For realworld data for descriptor calculation one of the most common used library is [Mordred Descriptors](https://github.com/mordred-descriptor/mordred) (~ 1500 descriptors)

# Substructure Search

This example shows how you can find specific substrates within a group of molecules. The creation of a SMARTS (SMiles ARbitrary Target Specification) pattern can help identify a substrate in order to help find it. SMARTS allows us to use more complex and flexible descriptions of molecular substructures. 

In this example specifically, let's break down what each piece of notation means:

1.	[*r6R1]1:
- [*]: Matches any atom.
- [r6]: Matches atoms that are part of a 6-membered ring.
- [R1]: Matches atoms that are part of exactly one ring.
- 1: Indicates the start of a ring, denoted by the number 1.
2.	[cR2]2:
- [c]: Matches aromatic carbon atoms.
- [R2]: Matches atoms that are part of exactly two rings.
- 2: Indicates the start of a second ring, denoted by the number 2.
3.	[cR1][cR1][cR1][cR1]:
- These are aromatic carbon atoms (c) that are part of exactly one ring (R1).
4.	[cR2]2:
- This is an aromatic carbon atom that is part of exactly two rings, closing the second ring started by 2.
5.	[*r6R1][*r6R1][*r6R1]1:
- These are any atoms that are part of a 6-membered ring and exactly one ring ([*r6R1]), closing the first ring started by 1.

In [ ]:
benzo_six = Chem.MolFromSmarts('[*r6R1]1[cR2]2[cR1][cR1][cR1][cR1][cR2]2[*r6R1][*r6R1][*r6R1]1')

In [ ]:
benzo_six

In [ ]:
data_small[data_small['ROMol'] >= benzo_six][['ROMol']]

In [ ]:
data_small["ROMol"].apply(lambda x: x.HasSubstructMatch(benzo_six) )

In [ ]:
data_small["HasBenzoSix"]=data_small["ROMol"].apply(lambda x: x.HasSubstructMatch(benzo_six) )

This code pattern matches a SMARTS string to other molecules and creates a grid of matching molecules 

In [ ]:

highlight_lists = [mol.GetSubstructMatch(benzo_six) for mol in data_small["ROMol"].to_list()]
Draw.MolsToGridImage(data_small["ROMol"].to_list(), 
                     highlightAtomLists = highlight_lists,
                     useSVG=False
                    )

Filters the molecules for the ones containing "HasBenzoSix"

In [ ]:
highlight_lists = [mol.GetSubstructMatch(benzo_six) for mol in data_small[data_small["HasBenzoSix"]]["ROMol"].to_list()]
Draw.MolsToGridImage(data_small[data_small["HasBenzoSix"]]["ROMol"].to_list(), 
                     highlightAtomLists = highlight_lists,
                     useSVG=False
                    )

In [ ]:
AllChem.Compute2DCoords(benzo_six)

In [ ]:
for mol in data_small[data_small["HasBenzoSix"]]["ROMol"].to_list():
    AllChem.GenerateDepictionMatching2DStructure(mol, benzo_six, acceptFailure = True)

In [ ]:
highlight_lists = [mol.GetSubstructMatch(benzo_six) for mol in data_small[data_small["HasBenzoSix"]]["ROMol"].to_list()]
Draw.MolsToGridImage(data_small[data_small["HasBenzoSix"]]["ROMol"].to_list(), 
                     highlightAtomLists = highlight_lists,
                     useSVG=False
                    )

## Murcko scaffolds

Murcko scaffolds are used to identify the core structure of a molecule, stripped away side chains and other features. This example shows how to generate and visualize one. 

In [ ]:
from rdkit.Chem.Scaffolds import MurckoScaffold
core = MurckoScaffold.GetScaffoldForMol(my_this_mol_obj)

In [ ]:
Draw.MolsToGridImage([core,my_this_mol_obj,my_that_mol_obj])

Here's what a Murcko Scaffold looks like as a SMILES string. 

In [ ]:
fw = MurckoScaffold.MakeScaffoldGeneric(core)
Chem.MolToSmiles(fw)


In [ ]:
Draw.MolsToGridImage([fw,core,my_this_mol_obj],legends=["generic scaffold","scaffold","molecule"])

In [ ]:
AllChem.Compute2DCoords(core)


In [ ]:
for m in [core,my_this_mol_obj]:
    AllChem.GenerateDepictionMatching2DStructure(m,core)

In [ ]:
Draw.MolsToGridImage([core,my_this_mol_obj],legends=["scaffold","molecule"])